<a href="https://colab.research.google.com/github/Mackenzzy/machinelearning/blob/main/estudoAprendizagemDeMaquina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#instalação da lib e abrindo seção no spark
!pip install pyspark==3.3.1
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .master('local[*]') \
    .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=6461c54eca16ec4a71464ab15a2068d1eabac1f3190624c31bf50e6e535760cd
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [2]:
#download do db sobre o covid, fiz separado pois é um db gigante

!gdown 1-lSo7A-SIuntho9tAqVO25enmLlxFe7S
!gdown 1-iG4e4-SzJONLi3Nw98nm9UdFzoqCHDI

Downloading...
From: https://drive.google.com/uc?id=1-lSo7A-SIuntho9tAqVO25enmLlxFe7S
To: /content/treinamento.csv
100% 26.1M/26.1M [00:00<00:00, 56.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-iG4e4-SzJONLi3Nw98nm9UdFzoqCHDI
To: /content/teste.csv
100% 37.4M/37.4M [00:00<00:00, 42.3MB/s]


In [3]:
# criando os dados para servir de treino
dfTreino = sc.read\
    .option('delimiter',',')\
    .option('header','true')\
    .option('inferschema','true')\
    .csv('file:////content/treinamento.csv')

# criando os dados para ser usados no teste
dfTeste = sc.read\
    .option('delimiter',',')\
    .option('header','true')\
    .option('inferschema','true')\
    .csv('file:////content/teste.csv')

In [4]:
#nome da coluna \ tipo do dado

dfTreino.printSchema()

root
 |-- hora: double (nullable = true)
 |-- minuto: double (nullable = true)
 |-- temp_minima: double (nullable = true)
 |-- temp_maxima: double (nullable = true)
 |-- latitude_media: double (nullable = true)
 |-- longitude_media: double (nullable = true)
 |-- Classe: string (nullable = true)



In [5]:
#lib para os id's
from pyspark.sql.functions import monotonically_increasing_id
#como a tabela não apresentou a coluna id(padrão no powerbi) eu adicionei
#para vir no indice 1 ao invés do 0 precisa ser colocado o '+1' depois dos ()
dfTeste = dfTeste.withColumn('id', monotonically_increasing_id() + 1)

#mostra as N primeiras linhas
dfTeste.show(8)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|11.787212|29.005116|      1.123|     28.606|      34.56035|      136.21983|Moderado|  1|
|11.482636| 29.84576|      1.038|  29.306004|      34.56078|      136.22015|Moderado|  2|
|11.312064|29.751745|      1.123|  29.006002|      34.55864|      136.22047|Moderado|  3|
|11.530982|29.505793|      1.404|  29.081003|     34.560654|      136.22006|Moderado|  4|
|  11.4008|29.422289|      1.504|  29.181004|     34.559814|      136.22006|Moderado|  5|
| 11.50784| 29.18513|      1.423|  29.106003|     34.559834|      136.21964|Moderado|  6|
|11.318713|29.743177|      1.323|  28.806002|     34.560326|       136.2207|Moderado|  7|
|11.324524| 30.05444|      1.155|  29.006002|     34.560883|      136.21967|Moderado|  8|
+---------

In [6]:
#mesma coisa acima porém para o df de treino
dfTreino = dfTreino.withColumn('id', monotonically_increasing_id() + 1)

#mostra as N primeiras linhas
dfTreino.show(8)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|11.312758|30.169239|     -1.859|     27.495|     36.169994|      139.23022|Moderado|  1|
|11.292323|29.638779|   8.542997|  36.177994|     31.349113|       73.50964|  Quente|  2|
|11.600304| 29.64275|     -1.861|  27.695002|      36.17158|      139.22937|Moderado|  3|
|11.462425|30.157314|   9.776998|  36.077995|     31.351593|        73.5104|  Quente|  4|
|11.735474|  29.9684|     -1.662|  27.695002|     36.169456|      139.23021|Moderado|  5|
|11.544413| 30.08596|  15.882999|  33.014996|     23.310606|      88.250885|  Alerta|  6|
|11.614842|29.549337|  11.020995|   36.58599|      26.52031|       78.81148|  Alerta|  7|
|11.699689|29.373444|  23.332996|  28.202002|      0.800029|      110.08771|  Alerta|  8|
+---------

In [7]:
from pyspark.ml.feature import StringIndexer, VectorAssembler  # Importa as classes StringIndexer e VectorAssembler do módulo de recursos do PySpark.

#colunas que a maquina vai estudar
features = ['hora', 'minuto', 'temp_minima', 'temp_maxima', 'latitude_media', 'longitude_media']

#usei o stringindexer pra transformar dados de string em numeros pra ficar mais facil para os algoritmos de aprendizagem de maquina
dfTreino = StringIndexer(inputCol='Classe', outputCol='label').fit(dfTreino).transform(dfTreino)

# nesse caso especifico não mesclei colunas, mas por convenção usei o vectorAssembler pra ficar mais facil a leitura
dfTreino = VectorAssembler(inputCols=features, outputCol='features')\
    .transform(dfTreino)

#exibe as 50 linhas iniciais
dfTreino.show(50)

#nao executar essa célula novamente, pois toda vez que rodar vai tentar criar as colunas novamente

+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|label|            features|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|11.312758|30.169239|     -1.859|     27.495|     36.169994|      139.23022|Moderado|  1|  0.0|[11.312758,30.169...|
|11.292323|29.638779|   8.542997|  36.177994|     31.349113|       73.50964|  Quente|  2|  3.0|[11.292323,29.638...|
|11.600304| 29.64275|     -1.861|  27.695002|      36.17158|      139.22937|Moderado|  3|  0.0|[11.600304,29.642...|
|11.462425|30.157314|   9.776998|  36.077995|     31.351593|        73.5104|  Quente|  4|  3.0|[11.462425,30.157...|
|11.735474|  29.9684|     -1.662|  27.695002|     36.169456|      139.23021|Moderado|  5|  0.0|[11.735474,29.968...|
|11.544413| 30.08596|  15.882999|  33.014996|     23.310606|    

In [8]:
dfTreino.show(5)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|label|            features|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|11.312758|30.169239|     -1.859|     27.495|     36.169994|      139.23022|Moderado|  1|  0.0|[11.312758,30.169...|
|11.292323|29.638779|   8.542997|  36.177994|     31.349113|       73.50964|  Quente|  2|  3.0|[11.292323,29.638...|
|11.600304| 29.64275|     -1.861|  27.695002|      36.17158|      139.22937|Moderado|  3|  0.0|[11.600304,29.642...|
|11.462425|30.157314|   9.776998|  36.077995|     31.351593|        73.5104|  Quente|  4|  3.0|[11.462425,30.157...|
|11.735474|  29.9684|     -1.662|  27.695002|     36.169456|      139.23021|Moderado|  5|  0.0|[11.735474,29.968...|
+---------+---------+-----------+-----------+--------------+----

In [ ]:
#se executar por engano as celuas acima :D
#dfTreino = dfTreino.drop('label')
#dfTreino = dfTreino.drop('features2')


In [9]:
# agora modelei o df de teste

from pyspark.ml.feature import StringIndexer, VectorAssembler

features = ['hora', 'minuto', 'temp_minima', 'temp_maxima', 'latitude_media', 'longitude_media']

dfTeste = StringIndexer(inputCol='Classe', outputCol='label')\
    .fit(dfTeste)\
    .transform(dfTeste)

dfTeste = VectorAssembler(inputCols=features, outputCol='features')\
    .transform(dfTeste)

In [10]:
dfTeste.show(5)

dfTreino.show(5)


+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|label|            features|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|11.787212|29.005116|      1.123|     28.606|      34.56035|      136.21983|Moderado|  1|  0.0|[11.787212,29.005...|
|11.482636| 29.84576|      1.038|  29.306004|      34.56078|      136.22015|Moderado|  2|  0.0|[11.482636,29.845...|
|11.312064|29.751745|      1.123|  29.006002|      34.55864|      136.22047|Moderado|  3|  0.0|[11.312064,29.751...|
|11.530982|29.505793|      1.404|  29.081003|     34.560654|      136.22006|Moderado|  4|  0.0|[11.530982,29.505...|
|  11.4008|29.422289|      1.504|  29.181004|     34.559814|      136.22006|Moderado|  5|  0.0|[11.4008,29.42228...|
+---------+---------+-----------+-----------+--------------+----

In [12]:
#calculando a acuracia de classificação na base de testes para o indicador de arvore de decisao


from pyspark.ml.classification import DecisionTreeClassifier
import pyspark.sql.functions as func
#usando as lib acima vamos criar uma instancia na arvore de decisao
dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')
#treina o modelo
modeloDT = dt.fit(dfTreino)

In [13]:
#faz as previsões do modelo de teste
predicaoDT = modeloDT.transform(dfTeste)
predicaoDT.select(func.col('prediction'), func.col('label'), func.col('id')).show(5)

+----------+-----+---+
|prediction|label| id|
+----------+-----+---+
|       0.0|  0.0|  1|
|       0.0|  0.0|  2|
|       0.0|  0.0|  3|
|       0.0|  0.0|  4|
|       0.0|  0.0|  5|
+----------+-----+---+
only showing top 5 rows



In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#confere e valida as colunas do modelo de treino e da precição
MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')\
    .evaluate(predicaoDT)

#seguindo a formula da acuracia

0.938528131874999

Extraindo dados do DB com aprendizagem de maquina



In [21]:
#proporção de falso negativo previstos pelo modelo
predicaoDT.select('label', 'prediction')\
.where(predicaoDT.label == 3.0)\
.where(predicaoDT.prediction != 3.0)\
.count() / predicaoDT.where(predicaoDT.label == 3.0).count()


0.14949778089231489

In [20]:
#numero absoluto de falsos negativos
falsos_negativos = predicaoDT.filter((func.col('label') == 3.0) & (func.col('prediction') != 3.0))
quantidade_falsos_negativos = falsos_negativos.count()

print(f'Quantidade "Alertas" classificados erroneamente como outra classe (falsos negativos): {quantidade_falsos_negativos}')

Quantidade "Alertas" classificados erroneamente como outra classe (falsos negativos): 11520


In [27]:
from pyspark.ml.classification import RandomForestClassifier
import pyspark.sql.functions as func

#cria a isntancia da rf com 5 arvores
rf = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=5)

#treina o modelo
modeloRF = rf.fit(dfTreino)

predicaoRF = modeloRF.transform(dfTeste)

predicaoRF.select(func.col('prediction'), func.col('label'), func.col('id')).show(5)

#proporção de falso negativos
proporcao_falsos_negativos = predicaoRF.select('label', 'prediction')\
    .where(predicaoRF.label == 3.0)\
    .where(predicaoRF.prediction != 3.0)\
    .count() / predicaoRF.where(predicaoRF.label == 3.0).count()

print(f'Proporção de falsos negativos: {proporcao_falsos_negativos}')

#numero total
falsos_negativos_rf = predicaoRF.filter((func.col('label') == 3.0) & (func.col('prediction') != 3.0))
quantidade_falsos_negativos_rf = falsos_negativos_rf.count()

print(f'Quantidade "Alertas" classificados erroneamente como outra classe (falsos negativos): {quantidade_falsos_negativos_rf}')

+----------+-----+---+
|prediction|label| id|
+----------+-----+---+
|       0.0|  0.0|  1|
|       0.0|  0.0|  2|
|       0.0|  0.0|  3|
|       0.0|  0.0|  4|
|       0.0|  0.0|  5|
+----------+-----+---+
only showing top 5 rows

Proporção de falsos negativos: 0.26676010277972434
Quantidade "Alertas" classificados erroneamente como outra classe (falsos negativos): 20556


In [28]:
from pyspark.ml.classification import RandomForestClassifier
import pyspark.sql.functions as func

rf = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=100)

# treina o modelo
modeloRF = rf.fit(dfTreino)

predicaoRF = modeloRF.transform(dfTeste)

predicaoRF.select(func.col('prediction'), func.col('label'), func.col('id')).show(5)

# proporção de falso negativo
proporcao_falsos_negativos = predicaoRF.select('label', 'prediction')\
    .where(predicaoRF.label == 3.0)\
    .where(predicaoRF.prediction != 3.0)\
    .count() / predicaoRF.where(predicaoRF.label == 3.0).count()

print(f'Proporção de falsos negativos: {proporcao_falsos_negativos}')

# numero total
falsos_negativos_rf = predicaoRF.filter((func.col('label') == 3.0) & (func.col('prediction') != 3.0))
quantidade_falsos_negativos_rf = falsos_negativos_rf.count()

print(f' "Alertas" classificados erroneamente como outra classe (falsos negativos): {quantidade_falsos_negativos_rf}')


+----------+-----+---+
|prediction|label| id|
+----------+-----+---+
|       0.0|  0.0|  1|
|       0.0|  0.0|  2|
|       0.0|  0.0|  3|
|       0.0|  0.0|  4|
|       0.0|  0.0|  5|
+----------+-----+---+
only showing top 5 rows

Proporção de falsos negativos: 0.19761737911702873
 "Alertas" classificados erroneamente como outra classe (falsos negativos): 15228
